In [1]:
from traitlets import Unicode, validate
from traittypes import Array
import numpy as np

import ipywidgets as widgets
from IPython.display import display


def array_to_json(ar, obj=None):
	return ar.tolist() if ar is not None else None

def from_json_to_array(value, obj=None):
	return np.array(value) if value else None


array_serialization = dict(to_json=array_to_json, from_json=from_json_to_array)

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    numpy_bigv = Array(default_value=None,allow_none=True).tag(sync=True)
    array_bigv = Array(default_value=None,allow_none=True).tag(sync=True,**array_serialization)
    

In [2]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed()
            
            this.model.on('change:value', this.value_changed, this);
        },

        value_changed: function() {
            this.el.textContent = this.model.get('numpy_bigv')[1];
        },
    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [3]:
from functools import update_wrapper
from ipykernel.jsonutil import *

def json_clean(obj):
    """Clean an object to ensure it's safe to encode in JSON.

    Atomic, immutable objects are returned unmodified.  Sets and tuples are
    converted to lists, lists are copied and dicts are also copied.

    Note: dicts whose keys could cause collisions upon encoding (such as a dict
    with both the number 1 and the string '1' as keys) will cause a ValueError
    to be raised.

    Parameters
    ----------
    obj : any python object

    Returns
    -------
    out : object

      A version of the input which will not cause an encoding error when
      encoded as JSON.  Note that this function does not *encode* its inputs,
      it simply sanitizes it so that there will be no encoding errors later.

    """
    # types that are 'atomic' and ok in json as-is.
    atomic_ok = (unicode_type, type(None))

    # containers that we need to convert into lists
    container_to_list = (tuple, set, types.GeneratorType)

    # Since bools are a subtype of Integrals, which are a subtype of Reals,
    # we have to check them in that order.

    if isinstance(obj, bool):
        return obj

    if isinstance(obj, numbers.Integral):
        # cast int to int, in case subclasses override __str__ (e.g. boost enum, #4598)
        return int(obj)

    if isinstance(obj, numbers.Real):
        # cast out-of-range floats to their reprs
        if math.isnan(obj) or math.isinf(obj):
            return repr(obj)
        return float(obj)
    
    if isinstance(obj, atomic_ok):
        return obj

    if isinstance(obj, bytes):
        return obj.decode(DEFAULT_ENCODING, 'replace')

    if isinstance(obj, container_to_list) or (
        hasattr(obj, '__iter__') and hasattr(obj, next_attr_name)):
        obj = list(obj)

    if isinstance(obj, list):
        return [json_clean(x) for x in obj]

    if isinstance(obj, dict):
        # First, validate that the dict won't lose data in conversion due to
        # key collisions after stringification.  This can happen with keys like
        # True and 'true' or 1 and '1', which collide in JSON.
        nkeys = len(obj)
        nkeys_collapsed = len(set(map(unicode_type, obj)))
        if nkeys != nkeys_collapsed:
            raise ValueError('dict cannot be safely converted to JSON: '
                             'key collision would lead to dropped values')
        # If all OK, proceed by making the new dict that will be json-safe
        out = {}
        for k,v in iteritems(obj):
            out[unicode_type(k)] = json_clean(v)
        return out
    if isinstance(obj, datetime):
        return obj.strftime(ISO8601)
    
    #################################
    #Here is the new part
    if isinstance(obj,np.ndarray):
        return obj.tolist()
    


In [4]:
Val = np.zeros(1000000)+0.1
#Val =  np.array([10,20])
from ipykernel.comm import comm

comm.json_clean = json_clean

%timeit A = HelloWidget(numpy_bigv=Val)


from ipykernel.jsonutil import json_clean
comm.json_clean = json_clean

%timeit A = HelloWidget(array_bigv=Val.tolist())


1 loop, best of 3: 348 ms per loop
1 loop, best of 3: 4.09 s per loop
